In [1]:
import psycopg

from pathlib import Path
from psycopg.rows import dict_row
from fishsense_api_workflow_worker.database import Database
from fishsense_api_workflow_worker.models.camera import Camera
from fishsense_api_workflow_worker.models.dive import Dive
from fishsense_api_workflow_worker.models.image import Image
from fishsense_api_workflow_worker.models.laser_label import LaserLabel
from fishsense_api_workflow_worker.models.head_tail_label import HeadTailLabel

In [2]:
FABRICANT_PASSWORD_PATH=Path("../.secrets/postgres_fabricant_password.txt")

with FABRICANT_PASSWORD_PATH.open() as f:
    fabricant_password = f.read().strip()

In [3]:
PG_CONN_STR = (
    f"postgres://postgres:{fabricant_password}@"
    f"fabricant-prod.ucsd.edu:5432/"
    f"postgres"
)

In [4]:
SELECT_CAMERAS_QUERY = "SELECT serial_number, name FROM cameras"

In [5]:
database = Database()
await database.init_database()

In [6]:
with psycopg.connect(PG_CONN_STR, row_factory=dict_row) as con, con.cursor() as cur:
    rows = cur.execute(query=SELECT_CAMERAS_QUERY)

    for row in rows:
        existing_camera = await database.select_camera_by_serial_number(row["serial_number"])
        if existing_camera is not None:
            print(existing_camera)
            continue

        camera = Camera(serial_number=row["serial_number"], name=row["name"])
        await database.insert_or_update_camera(camera)

In [7]:
SELECT_DIVES_QUERY = "SELECT path, date FROM dives ORDER BY date"
SELECT_IMAGES_QUERY = "SELECT path, dive, date, image_md5, camera_sn FROM images ORDER BY date"

In [8]:
dives = []
images = []

with psycopg.connect(PG_CONN_STR, row_factory=dict_row) as con, con.cursor() as cur:
    dive_rows = cur.execute(query=SELECT_DIVES_QUERY)
    for dive_row in dive_rows:
        if ".Trashes" in dive_row["path"]:
            continue

        dives.append(dive_row)

    image_rows = cur.execute(query=SELECT_IMAGES_QUERY)
    for image_row in image_rows:
        images.append(image_row)

QueryCanceled: canceling statement due to statement timeout

In [ ]:
for dive in dives:
    existing_dive = await database.select_dive_by_path(dive["path"])
    if existing_dive is not None:
        print(existing_dive)
        continue

    dive_images = [i for i in images if i["dive"] == dive["path"]]
    image_dates = [i["date"] for i in dive_images]

    image_dates.sort()
    end_date = image_dates[-1]

    camera = await database.select_camera_by_serial_number(images[-1]["camera_sn"])

    dive = Dive(path=dive["path"], dive_datetime=end_date, camera_id=camera.id)
    await database.insert_or_update_dive(dive)

dive_datetime=datetime.datetime(2023, 8, 1, 12, 46, 27) id=1 camera_id=4 priority=<Priority.LOW: 'LOW'> path='2023-09-07 REEF Data Dump/080123_FSL-01 Photos'
dive_datetime=datetime.datetime(2023, 8, 1, 12, 46, 27) id=2 camera_id=4 priority=<Priority.LOW: 'LOW'> path='2024.06.20.REEF/08_2023/080123_FSL-01 Photos'
dive_datetime=datetime.datetime(2023, 8, 1, 12, 46, 27) id=3 camera_id=4 priority=<Priority.LOW: 'LOW'> path='2025-02-10 REEF Data Dump SMILE 6/080123_FSL-01 Photos'
dive_datetime=datetime.datetime(2023, 8, 3, 9, 43, 11) id=4 camera_id=4 priority=<Priority.LOW: 'LOW'> path='2023.08.03.FishSense.FSL-01D/Hogfish02_MolHITW_0943_080323'
dive_datetime=datetime.datetime(2023, 8, 3, 9, 27, 23) id=5 camera_id=4 priority=<Priority.LOW: 'LOW'> path='2023.08.03.FishSense.FSL-01D/Hogfish01_MolHITW_0926_080323'
dive_datetime=datetime.datetime(2023, 8, 3, 10, 29, 49) id=6 camera_id=4 priority=<Priority.LOW: 'LOW'> path='2023.08.03.FishSense.FSL-01D/Hogfish01_MolPeLe_1029_080323'
dive_datetim

In [ ]:
dives = await database.select_dives()

for dive in dives:
    images_per_dive = [i for i in images if i["dive"] == dive.path]
    images_per_dive.sort(key=lambda i: i["date"])

    for image_row in images_per_dive:
        existing_image = await database.select_image_by_path(image_row["path"])

        if existing_image is not None:
            continue

        existing_checksum = await database.select_image_by_checksum(image_row["image_md5"])

        camera = await database.select_camera_by_serial_number(image_row["camera_sn"])
        image = Image(path=image_row["path"], taken_datetime=image_row["date"], checksum=image_row["image_md5"], dive_id=dive.id, camera_id=camera.id, is_canonical=existing_checksum is None)
        await database.insert_or_update_image(image)

In [ ]:
SELECT_LASER_LABELS_QUERY = "SELECT cksum, task_id, x, y FROM laser_labels"

In [ ]:
with psycopg.connect(PG_CONN_STR, row_factory=dict_row) as con, con.cursor() as cur:
    rows = cur.execute(query=SELECT_LASER_LABELS_QUERY)

    for row in rows:
        existing_label = await database.select_laser_label_by_task_id(row["task_id"])
        if existing_label is not None:
            print(existing_label)
            continue

        image = await database.select_image_by_checksum(row["cksum"])

        label = LaserLabel(label_studio_task_id=row["task_id"], x=row["x"], y=row["y"], image_id=image.id)
        await database.insert_or_update_laser_label(label)

In [ ]:
SELECT_CANONICAL_DIVES_QUERY = "SELECT path, priority FROM canonical_dives"

In [ ]:
with psycopg.connect(PG_CONN_STR, row_factory=dict_row) as con, con.cursor() as cur:
    rows = cur.execute(query=SELECT_CANONICAL_DIVES_QUERY)

    for row in rows:
        dive = await database.select_dive_by_path(row["path"])
        if dive is None:
            print(f"Could not find dive for path {row['path']}")
            continue

        dive.priority = row["priority"]

        await database.insert_or_update_dive(dive)

Could not find dive for path drive-download-20240307T1050Z/.Trashes/501
Could not find dive for path drive-download-20240307T1050Z/.Trashes/501/082923_Pool Calibration/082323_Slate_FSL01
Could not find dive for path drive-download-20240307T1050Z/.Trashes/501/082923_Pool Calibration/082323_Slate_FSL04
Could not find dive for path drive-download-20240307T1050Z/.Trashes/501/082923_Pool Calibration/082329_Slate_FSL05


In [ ]:
SELECT_HEAD_TAIL_LABELS_QUERY = "SELECT cksum, task_id, head_x, head_y, tail_x, tail_y FROM headtail_labels"

In [ ]:
with psycopg.connect(PG_CONN_STR, row_factory=dict_row) as con, con.cursor() as cur:
    rows = cur.execute(query=SELECT_HEAD_TAIL_LABELS_QUERY)

    for row in rows:
        existing_label = await database.select_head_tail_labels_by_task_id(row["task_id"])
        if existing_label is not None:
            print(existing_label)
            continue

        image = await database.select_image_by_checksum(row["cksum"])

        label = HeadTailLabel(label_studio_task_id=row["task_id"], head_x=row["head_x"], head_y=row["head_y"], tail_x=row["tail_x"], tail_y=row["tail_y"], image_id=image.id)
        await database.insert_or_update_head_tail_label(label)